In [44]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.datasets import make_classification
from sklearn.base import clone

In [45]:
X, y = make_classification(n_samples=1000, n_features = 8, n_classes = 2)

df = pd.DataFrame(X)
df['target'] = y
print(df.shape)
df.head()

(1000, 9)


,0,1,2,3,4,5,6,7,target
0,-1.008698,-0.817196,-0.100143,0.548948,0.336592,1.626982,-0.950899,0.752413,1
1,2.415784,2.547514,2.005128,-0.853887,1.758098,-0.878500,-0.415049,0.276686,1
2,1.061579,0.799373,-0.076002,0.560392,0.798482,-0.898551,0.184060,-1.005456,0
3,-2.201794,-1.958033,-0.739630,2.036347,-1.009145,-0.004268,-0.538177,1.028838,1
4,0.244002,-0.403510,-1.773412,-0.095712,-1.304864,-0.472409,-1.067892,-2.298776,0


In [50]:
target = 'target'

cat = CatBoostClassifier(n_estimators= 100, verbose = False, allow_writing_files=False)
xgb = XGBClassifier(n_estimators= 100)

X = df.drop(columns = [target], axis = 1)
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y)

print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

X_train shape: (750, 8)
X_test shape: (250, 8)
y_train shape: (750,)
y_test shape: (250,)


In [93]:
cat.fit(X_train, y_train)
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [108]:
class AveragingModels:
    def __init__(self, models):
        self.models = models
        self.scaler = MinMaxScaler()

    def fit(self, X, y, eval_set=None, early_stopping_rounds=None, verbose=None):
        self.feature_names_ = X.columns
        self.models_ = [clone(x) for x in self.models]
        for model in self.models_:
            model.fit(
                X,
                y,
                eval_set=eval_set,
                early_stopping_rounds=early_stopping_rounds,
                verbose=verbose,
            )

        importance = np.zeros(len(self.feature_names_))
        for model in self.models_:
            importance += model.feature_importances_ / len(self.models_)

        self.feature_importances_ = importance
        return self

    def predict(self, X):

        predictions = np.array([model.predict_proba(X) for model in self.models_]) / len(self.models_)

        predictions
        
        return 

In [109]:
avg_models = AveragingModels(models = (xgb, cat))

avg_models.fit(X_train, y_train, verbose=False)

y_pred = avg_models.predict(X_test)

print(f'avg models accuracy: {accuracy_score(y_test, y_pred)}')

ValueError: Found input variables with inconsistent numbers of samples: [250, 2]

In [110]:
y_pred

array([[[0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        

In [61]:
y_test.shape

(250,)